# Char RNN

## Summary

In this notebook, I have created an Char-rnn model to write a story.

In [28]:
from __future__ import print_function
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils
import numpy as np

In [103]:
raw_text = open('C:/Users/Shaurya/summer/keras/wonderland.txt').read()
raw_text = raw_text.lower()
print (raw_text[:200])

alice's adventures in wonderland

lewis carroll

the millennium fulcrum edition 3.0

chapter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on the
bank, and of


In [104]:
chars = sorted(list(set(raw_text)))
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters:", n_chars)
print ("Total Vocab:", n_vocab)

Total Characters: 144413
Total Vocab: 45


In [105]:
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [106]:
seq_len = 10
step = 1

inp_chars=[]
out_chars=[]

for i in range(0, (n_chars-seq_len), step):
    inp_chars.append(raw_text[i:i+seq_len])
    out_chars.append(raw_text[i+seq_len])

In [107]:
for j in range(5):
    print(inp_chars[j] + ' -> ' + out_chars[j])

alice's ad -> v
lice's adv -> e
ice's adve -> n
ce's adven -> t
e's advent -> u


In [108]:
print('Total patterns: ',len(inp_chars))

Total patterns:  144403


In [109]:
X=[]
y=[]

for k in range(len(inp_chars)):
    X.append([char_to_int[l] for l in inp_chars[k]])
    y.append([char_to_int[m] for m in out_chars[k]])

In [110]:
X[0]

[19, 30, 27, 21, 23, 4, 37, 1, 19, 22]

In [111]:
# Reshape dataX to size of [samples, time steps, features] and scale it to 0-1
# Represent dataY as one hot encoding
X_train = np.reshape(X, (len(X), seq_len, 1))/float(n_vocab)
y_train = np_utils.to_categorical(y)

In [114]:
y_train.shape

(144403, 45)

In [115]:
X_train.shape

(144403, 10, 1)

In [40]:
model = Sequential()
model.add(SimpleRNN(128, return_sequences=False,input_shape=( seq_len, 1)))
model.add(Dense(y_train.shape[1]))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_5 (SimpleRNN)     (None, 128)               16640     
_________________________________________________________________
dense_5 (Dense)              (None, 45)                5805      
_________________________________________________________________
activation_5 (Activation)    (None, 45)                0         
Total params: 22,445.0
Trainable params: 22,445.0
Non-trainable params: 0.0
_________________________________________________________________


In [42]:
model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics=['accuracy'])

In [43]:
model.fit(X_train, y_train, epochs=30, batch_size=64, validation_split= 0.2)

Train on 115522 samples, validate on 28881 samples
Epoch 1/30
115522/115522 [==============================] - 23s - loss: 2.9149 - acc: 0.1974 - val_loss: 2.8551 - val_acc: 0.2197
Epoch 2/30
115522/115522 [==============================] - 22s - loss: 2.7834 - acc: 0.2281 - val_loss: 2.7666 - val_acc: 0.2364
Epoch 3/30
115522/115522 [==============================] - 22s - loss: 2.7129 - acc: 0.2425 - val_loss: 2.7072 - val_acc: 0.2488
Epoch 4/30
115522/115522 [==============================] - 22s - loss: 2.6654 - acc: 0.2548 - val_loss: 2.6758 - val_acc: 0.2511
Epoch 5/30
115522/115522 [==============================] - 22s - loss: 2.6273 - acc: 0.2627 - val_loss: 2.6411 - val_acc: 0.2549
Epoch 6/30
115522/115522 [==============================] - 22s - loss: 2.5940 - acc: 0.2679 - val_loss: 2.6299 - val_acc: 0.2646
Epoch 7/30
115522/115522 [==============================] - 25s - loss: 2.5639 - acc: 0.2737 - val_loss: 2.5931 - val_acc: 0.2732
Epoch 8/30
115522/115522 [=============

In [84]:
import sys

start = np.random.randint(0, len(X_train)-1)
pattern = X[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print ("\nGenerated Sequence:")
print(pattern)
for i in range(10):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/float(n_vocab)
    prediction = model.predict(x)
    index = np.argmax(prediction)
    print(index)
    result = int_to_char[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
    print(pattern)
print ("\nDone.")

Seed:
" rge kitche "

Generated Sequence:
[36, 25, 23, 1, 29, 27, 38, 21, 26, 23]
25
g[25, 23, 1, 29, 27, 38, 21, 26, 23, 25]
1
 [23, 1, 29, 27, 38, 21, 26, 23, 25, 1]
38
t[1, 29, 27, 38, 21, 26, 23, 25, 1, 38]
26
h[29, 27, 38, 21, 26, 23, 25, 1, 38, 26]
23
e[27, 38, 21, 26, 23, 25, 1, 38, 26, 23]
1
 [38, 21, 26, 23, 25, 1, 38, 26, 23, 1]
25
g[21, 26, 23, 25, 1, 38, 26, 23, 1, 25]
33
o[26, 23, 25, 1, 38, 26, 23, 1, 25, 33]
33
o[23, 25, 1, 38, 26, 23, 1, 25, 33, 33]
37
s[25, 1, 38, 26, 23, 1, 25, 33, 33, 37]

Done.


In [86]:
start = np.random.randint(0, len(X_train)-1)
pattern = X[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print ("\nGenerated Sequence:")
#print(pattern)
for i in range(50):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/float(n_vocab)
    prediction = model.predict(x)
    index = np.argmax(prediction)
    #print(index)
    result = int_to_char[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
   # print(pattern)
print ("\nDone.")

Seed:
" ir
doesn't "

Generated Sequence:
 kev the whnt to she whst ane she whnt to she whst
Done.


---------------------------

In [116]:
X_train2 = np.reshape(X, (len(X), seq_len))/float(n_vocab)

In [117]:
from keras.layers.embeddings import Embedding

In [122]:
embedding_vector_length = 32
model2 = Sequential()
model2.add(Embedding(n_vocab, embedding_vector_length, input_length=seq_len))
model2.add(LSTM(256))
model2.add(Dense(y_train.shape[1], activation='softmax'))
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 32)            1440      
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               295936    
_________________________________________________________________
dense_7 (Dense)              (None, 45)                11565     
Total params: 308,941.0
Trainable params: 308,941
Non-trainable params: 0.0
_________________________________________________________________


In [123]:
model2.compile(loss='categorical_crossentropy', optimizer='adam')

In [124]:
model2.fit(X_train2, y_train, epochs=10, batch_size=64, validation_split= 0.2)

Train on 115522 samples, validate on 28881 samples
Epoch 1/10
115522/115522 [==============================] - 197s - loss: 3.0404 - val_loss: 3.0514:
Epoch 2/10
115522/115522 [==============================] - 197s - loss: 3.0299 - val_loss: 3.0455
Epoch 3/10
115522/115522 [==============================] - 187s - loss: 3.0286 - val_loss: 3.0472
Epoch 4/10
115522/115522 [==============================] - 194s - loss: 3.0281 - val_loss: 3.0460
Epoch 5/10
115522/115522 [==============================] - 198s - loss: 3.0276 - val_loss: 3.0458s 
Epoch 6/10
115522/115522 [==============================] - 190s - loss: 3.0269 - val_loss: 3.0497
Epoch 7/10
115522/115522 [==============================] - 182s - loss: 3.0269 - val_loss: 3.0478
Epoch 8/10
115522/115522 [==============================] - 181s - loss: 3.0268 - val_loss: 3.0484
Epoch 9/10
115522/115522 [==============================] - 181s - loss: 3.0268 - val_loss: 3.0455
Epoch 10/10
115522/115522 [============================

In [137]:
start = np.random.randint(0, len(X_train)-1)
pattern = X[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print ("\nGenerated Sequence:")
#print(pattern)
for i in range(1):
    x = np.reshape(pattern, (1, len(pattern)))
    x = x/float(n_vocab)
    prediction = model2.predict(x)
    index = np.argmax(prediction)
    print(index)
    result = int_to_char[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
   # print(pattern)
print ("\nDone.")

Seed:
" s:
you'd b "

Generated Sequence:
1
 
Done.


In [156]:
start = np.random.randint(0, len(X_train)-1)
pattern = X[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
for i in range(5):
    x = np.reshape(pattern, (1, len(pattern)))
    x = x/float(n_vocab)
    prediction = model2.predict(x)
    index = np.argmax(prediction)
    result = int_to_char[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
    print(pattern)
print ("\nDone.")

Seed:
"  young lad "
1
 [43, 33, 39, 32, 25, 1, 30, 19, 22, 1]
1
 [33, 39, 32, 25, 1, 30, 19, 22, 1, 1]
1
 [39, 32, 25, 1, 30, 19, 22, 1, 1, 1]
1
 [32, 25, 1, 30, 19, 22, 1, 1, 1, 1]
1
 [25, 1, 30, 19, 22, 1, 1, 1, 1, 1]

Done.
